In [4]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import time
import json
import ast
from tqdm import tqdm
import requests
import random
import csv
from faker import Faker
import warnings
warnings.filterwarnings('ignore') # turn off warnings

import psycopg2


connUAT = psycopg2.connect(
     host='localhost',
     database='uat',
     port=5431,
     user='postgres',
     password='1234567890'
  )

connPROC = psycopg2.connect(
     host='localhost',
     database='prod',
     port=5431,
     user='postgres',
     password='1234567890'
  )

cursorUAT = connUAT.cursor()
cursorPROC = connPROC.cursor()

def selectdf(query,conn):
    df = pd.read_sql(query, conn)
    return df
  
def disconnect(cursor,conn):
    try:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
        print("Disconnected from database.")
    except Exception as e:
        print(f"Lỗi khi đóng kết nối: {e}")   
          
def SQL_push(sql_table_name,python_table,conn,cursor,inplace):
    python_table.replace([np.nan], [None],inplace=True)
    
    if inplace == True:
        try:
            sql_query = f'truncate table {sql_table_name}'
            cursor.execute(sql_query)
            conn.commit()
        except:
            pass
            
    sql = f"INSERT INTO {sql_table_name} VALUES ({','.join(['%s'] * len(python_table.columns))})"

    data = [tuple(row) for _, row in tqdm(python_table.iterrows(), total=len(python_table), desc="Preparing data")]

    # Chèn dữ liệu với xử lý lỗi
    for row in tqdm(data, total=len(data), desc="Inserting rows"):
        try:
            cursor.execute(sql, row)
        except Exception as e:
            print(f"Lỗi khi chèn dòng {row}: {e}")
            conn.rollback()  # Hủy giao dịch khi có lỗi, tránh PostgreSQL khóa transaction

    # Sau khi xong thì commit lại
    connUAT.commit()
    print("PUSH DATA: DONE")

pathfolder = r'F:\Work\Caohoc_2024_2026\PTPM_project\ml-latest'

In [2]:
# disconnect(cursorUAT,connUAT)

## Links


In [ ]:
def primary_key_take(sql_schema,sql_table_name):
    q = f'''
            SELECT
                a.attname AS column_name
            FROM
                pg_index i
            JOIN
                pg_attribute a ON a.attrelid = i.indrelid AND a.attnum = ANY(i.indkey)
            JOIN
                pg_class c ON c.oid = i.indrelid
            JOIN
                pg_namespace n ON n.oid = c.relnamespace
            WHERE
                i.indisprimary = true
                AND c.relname = {sql_schema}
                AND n.nspname = {sql_table_name};
        '''
    column_name = coreproc.selectdf(q)
    return column_name

def sql_append_check(sql_table_name,python_table):
    pk = coreproc.primary_key_take(sql_table_name)
    q = f'''select distinct {pk} from {sql_table_name}'''
    sqldf = coreproc.selectdf(q)
    sqlkeyid = set(sqldf[f'{pk}'])
    pythonkeyid = set(python_table[f'{pk}'])
    updatelist = list(pythonkeyid - sqlkeyid)
    updatedf = python_table[python_table[f'{pk}'].isin(updatelist)]
    return updatedf 

In [98]:
links = pd.read_csv(os.path.join(pathfolder,'links.csv'))
links = links[links['tmdbId'].isnull()==False]
links.sort_values(by='tmdbId',inplace=True)
links['keyid'] = links['movieId'].astype('int').astype('str') + links['imdbId'].astype('int').astype('str') + links['tmdbId'].astype('int').astype('str')
links = links[['keyid','movieId','imdbId','tmdbId']]
links.sort_values(by='tmdbId',inplace=True)
# links = links.head(20000)

In [4]:
a1 = links.groupby('tmdbId').agg(idcount = ('tmdbId','count')).reset_index()
a1.sort_values(by='idcount',ascending=False,inplace=True)
a1 = a1[a1['idcount']> 1]

In [5]:
links_check = links[links['tmdbId'].isin(a1['tmdbId'])].sort_values(by='tmdbId')
links_check

,keyid,movieId,imdbId,tmdbId
34363,1446062702884912,144606,270288,4912.0
5892,60032905384912,6003,290538,4912.0
10103,343303680899775,34330,368089,9775.0
49997,1787553768009775,178755,376800,9775.0
48013,17453323567910991,174533,235679,10991.0
...,...,...,...,...
76772,25410014513892642732,254100,14513892,642732.0
68897,22158610394706703771,221586,10394706,703771.0
78451,26188312876132703771,261883,12876132,703771.0
74220,24210013323128792517,242100,13323128,792517.0


In [6]:
links['imdbId'].duplicated().sum()

0

In [7]:
for i in ['movieId','imdbId','tmdbId']:
    print(f'{i} duplicate = {links[i].duplicated().sum()}')

movieId duplicate = 0
imdbId duplicate = 0
tmdbId duplicate = 37


## Ratings

In [7]:
movies_ratings = pd.read_csv(os.path.join(pathfolder,'ratings.csv'))
movies_ratings['keyid'] = movies_ratings['userId'].astype('str') + \
                          movies_ratings['movieId'].astype('str') + \
                          movies_ratings['timestamp'].astype('str')
movies_ratings['date_rate'] = pd.to_datetime(movies_ratings['timestamp'],unit='s')
del movies_ratings['timestamp']
movies_ratings.sort_values(by=['date_rate','movieId','userId'],inplace=True)
movies_ratings = movies_ratings[['keyid','userId','movieId','rating','date_rate']]
# movies_ratings = movies_ratings[movies_ratings['movieId'].isin(stg_links_list)]
movies_ratings.rename(columns={'movieId':'movieid','userId':'userid'},inplace=True)
movies_ratings

,keyid,userid,movieid,rating,date_rate
15254009,1499541176789652004,149954,1176,4.0,1995-01-09 11:46:44
9617113,9453221789652009,94532,21,3.0,1995-01-09 11:46:49
9617119,9453247789652009,94532,47,5.0,1995-01-09 11:46:49
9617162,945321079789652009,94532,1079,3.0,1995-01-09 11:46:49
16958922,1664761822873600,166476,1,4.0,1996-01-29 00:00:00
...,...,...,...,...,...
22026732,2148312818641689841440,214831,281864,1.5,2023-07-20 08:24:00
29836322,29155452821689841759,291554,5282,3.5,2023-07-20 08:29:19
29836181,29155412011689842491,291554,1201,4.5,2023-07-20 08:41:31
29836625,2915542151731689842596,291554,215173,4.5,2023-07-20 08:43:16


In [5]:
for i in ['keyid']:
    print(f'{i} duplicate = {movies_ratings[i].duplicated().sum()}')

keyid duplicate = 0


# Userid

In [9]:
# Khởi tạo Faker với ngôn ngữ mặc định (tiếng Anh)
fake = Faker()

# Tạo danh sách người dùng
num_users = len(set(movies_ratings['userid']))
users = []

for user_id in range(1, num_users + 1):
    name = fake.name()
    age = random.randint(18, 65)
    address = fake.address().replace("\n", ", ")
    # email = fake.email()
    users.append({
        'user_id': user_id,
        'name': name,
        'age': age,
        'address': address,
        # 'email': email
    })

# In thử 5 dòng đầu tiên
for user in users[:5]:
    print(user)

# Ghi ra file CSV
with open(os.path.join(pathfolder,'users_inf.csv'), 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['user_id', 'name', 'age', 'address']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for user in users:
        writer.writerow(user)

print(f"Done! {num_users} English user records saved to 'users_en.csv'")

{'user_id': 1, 'name': 'Christine Miller', 'age': 42, 'address': '0847 Jared Keys Apt. 610, Lake Saraborough, KS 56655'}
{'user_id': 2, 'name': 'Scott Jensen', 'age': 61, 'address': '29959 Santos Highway, Gonzalesmouth, MH 05874'}
{'user_id': 3, 'name': 'Kristin Branch', 'age': 34, 'address': '04157 Jessica Street Apt. 556, New Steven, MP 09765'}
{'user_id': 4, 'name': 'Bobby Cunningham', 'age': 35, 'address': '78555 Larson Road Suite 231, Lindsayside, WY 65912'}
{'user_id': 5, 'name': 'David Edwards', 'age': 42, 'address': '56695 Eric Court, South Jayfurt, MN 98360'}
Done! 330975 English user records saved to 'users_en.csv'


In [10]:
userinf = pd.read_csv(os.path.join(pathfolder,'users_inf.csv'))
userinf

,user_id,name,age,address
0,1,Christine Miller,42,"0847 Jared Keys Apt. 610, Lake Saraborough, KS..."
1,2,Scott Jensen,61,"29959 Santos Highway, Gonzalesmouth, MH 05874"
2,3,Kristin Branch,34,"04157 Jessica Street Apt. 556, New Steven, MP ..."
3,4,Bobby Cunningham,35,"78555 Larson Road Suite 231, Lindsayside, WY 6..."
4,5,David Edwards,42,"56695 Eric Court, South Jayfurt, MN 98360"
...,...,...,...,...
330970,330971,James Estes,23,"Unit 5550 Box 4089, DPO AE 16438"
330971,330972,Candace Francis,46,"7737 Jonathan Plaza Suite 165, Andreahaven, AZ..."
330972,330973,April Watson,34,"164 Davis Prairie, South Hannah, NM 13082"
330973,330974,Maria Livingston,64,"7755 Stone Ramp, South Maryside, PA 85346"


# Keywords

In [10]:
q = '''
        select distinct movieid
        from staging.stg_links
        order by movieid 
    '''

stg_links = selectdf(q,connUAT)
stg_links_list = list(set(stg_links['movieid'].values))
stg_links_list.sort()

pathfolder = r'F:\Work\Caohoc_2024_2026\PTPM_project\ml-latest'

moives_tags = pd.read_csv(os.path.join(pathfolder,'tags.csv'))
moives_tags.sort_values(by=['timestamp','movieId','userId'],inplace=True)
moives_tags.reset_index(drop=True,inplace=True)
moives_tags.reset_index(inplace=True)
moives_tags['date_tag'] = pd.to_datetime(moives_tags['timestamp'],unit='s')
moives_tags['tag'] = moives_tags['tag'].str.lower().str.strip()
moives_tags['tag'] = moives_tags['tag'].replace('',np.nan)
moives_tags.rename(columns={'tag':'keywords'},inplace=True)
moives_tags['keyid'] = moives_tags['userId'].astype('int').astype('str') + \
                       moives_tags['movieId'].astype('int').astype('str') + \
                       moives_tags['timestamp'].astype('int').astype('str') + \
                       moives_tags.index.astype('str')    
moives_tags = moives_tags[['keyid','userId','movieId','keywords','date_tag']]
moives_tags = moives_tags[moives_tags['movieId'].isin(stg_links_list)]
moives_tags

,keyid,userId,movieId,keywords,date_tag
0,83124278811354292100,83124,2788,monty python,2005-12-24 13:00:10
1,83124173211354292361,83124,1732,coen brothers,2005-12-24 13:00:36
2,83124120611354292482,83124,1206,stanley kubrick,2005-12-24 13:00:48
3,83124119311354293713,83124,1193,jack nicholson,2005-12-24 13:02:51
4,83124500411354293994,83124,5004,peter sellers,2005-12-24 13:03:19
...,...,...,...,...,...
2328310,12853610487916898407442328310,128536,104879,snakes,2023-07-20 08:12:24
2328311,12853610487916898407482328311,128536,104879,candlelight vigil,2023-07-20 08:12:28
2328312,12853610487916898407502328312,128536,104879,maze,2023-07-20 08:12:30
2328313,755966306216898412952328313,75596,63062,jms,2023-07-20 08:21:35


In [11]:
for i in ['keyid']:
    print(f'{i} duplicate = {moives_tags[i].duplicated().sum()}')

keyid duplicate = 0


# Metadata

In [2]:
sql_check = '''
            select distinct *
            from staging.stg_tmdb_json_movie_metadata stjmm 
            where request_json <> '{"success": false, "status_code": 34, "status_message": "The resource you requested could not be found."}'
            order by id
            '''
movie_json = selectdf(sql_check,connUAT)
# list_c['id'] = list_c['id'].astype(str)  # Đảm bảo cùng kiểu dữ liệu
movie_json.sort_values(by='id',inplace=True)

data_dict_list = []

for i in tqdm(set(movie_json['id'])):
    data_str = movie_json[movie_json['id']==i]['request_json'].values[0]
    data = f'''{data_str}'''
    data_dict = json.loads(data)
    data_dict_list.append(data_dict)
    
movie_metadata = pd.DataFrame(data_dict_list)
movie_metadata.sort_values(by='id',inplace=True)
movie_metadata


100%|██████████| 85304/85304 [00:24<00:00, 3422.66it/s]


,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,origin_country,original_language,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
1,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,"{'id': 1382526, 'name': 'Aki Kaurismäki's Prol...",0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",None,2,tt0094675,[FI],fi,...,1988-10-21,0,73,"[{'english_name': 'Finnish', 'iso_639_1': 'fi'...",Released,None,Ariel,False,7.129,345
2,False,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,"{'id': 1382526, 'name': 'Aki Kaurismäki's Prol...",0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",None,3,tt0092149,[FI],fi,...,1986-10-17,0,74,"[{'english_name': 'Finnish', 'iso_639_1': 'fi'...",Released,None,Shadows in Paradise,False,7.306,407
3,False,/f2t4JbUvQIjUF5FstG1zZFAp02N.jpg,None,4000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...",https://www.miramax.com/movie/four-rooms/,5,tt0113101,[US],en,...,1995-12-09,4257354,98,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,5.900,2683
4,False,/iUUpKunmBN5l8goObADBaFHnxQ8.jpg,None,21000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",None,6,tt0107286,[US],en,...,1993-10-15,12136938,109,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,False,6.500,346
5,False,/2w4xG178RpB4MDAIfTkqAuSJzec.jpg,"{'id': 10, 'name': 'Star Wars Collection', 'po...",11000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.starwars.com/films/star-wars-episod...,11,tt0076759,[US],en,...,1977-05-25,775398007,121,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"A long time ago in a galaxy far, far away...",Star Wars,False,8.203,21056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49780,False,None,None,0,[],None,1144338,None,[],en,...,None,0,0,[],Canceled,None,Table,False,0.000,0
50074,False,/4OIJmgez12DITjHbWiNf1C7uG1d.jpg,None,0,"[{'id': 37, 'name': 'Western'}]",None,1145358,tt23469212,[US],en,...,2023-07-07,0,96,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Play your cards right and you just might survive.,Dead Man's Hand,False,5.778,9
50369,False,/t86SkVd35W9COtCGtbxxb3UoQpW.jpg,None,0,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",None,1146246,tt6810618,[CA],en,...,2024-08-22,0,84,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,None,The Deep Web: Murdershow,False,4.600,5
50371,False,/eNopYAzvtfvaj1yCI0AeVAOlHzU.jpg,None,0,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",None,1146249,tt17519798,[US],en,...,2023-07-12,0,88,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Insane in the membrane.,Cabin Girl,False,5.100,19


In [13]:
movie_metadata.columns.to_list()

['adult',
 'backdrop_path',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'origin_country',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

In [14]:
for i in ['id','imdb_id']:
    print(f'{i} duplicate = {movie_metadata[i].duplicated().sum()}')

id duplicate = 0
imdb_id duplicate = 19


In [15]:
movie_metadata2 = movie_metadata[['id','title','original_title','original_language','release_date','status','overview','tagline','adult','popularity','homepage','poster_path','runtime','budget','revenue','vote_average','vote_count']]
movie_metadata2.head(5)

,id,title,original_title,original_language,release_date,status,overview,tagline,adult,popularity,homepage,poster_path,runtime,budget,revenue,vote_average,vote_count
1,2,Ariel,Ariel,fi,1988-10-21,Released,A Finnish man goes to the city to find a job a...,None,False,1.5038,None,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg,73,0,0,7.129,345
2,3,Shadows in Paradise,Varjoja paratiisissa,fi,1986-10-17,Released,"Nikander, a rubbish collector and would-be ent...",None,False,1.1210,None,/nj01hspawPof0mJmlgfjuLyJuRN.jpg,74,0,0,7.306,407
3,5,Four Rooms,Four Rooms,en,1995-12-09,Released,It's Ted the Bellhop's first night on the job....,Twelve outrageous guests. Four scandalous requ...,False,3.1509,https://www.miramax.com/movie/four-rooms/,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,98,4000000,4257354,5.900,2683
4,6,Judgment Night,Judgment Night,en,1993-10-15,Released,"Four young friends, while taking a shortcut en...",Don't move. Don't whisper. Don't even breathe.,False,1.1798,None,/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg,109,21000000,12136938,6.500,346
5,11,Star Wars,Star Wars,en,1977-05-25,Released,Princess Leia is captured and held hostage by ...,"A long time ago in a galaxy far, far away...",False,19.8534,http://www.starwars.com/films/star-wars-episod...,/6FfCtAuVAW8XJjZ7eWeLibRLWTw.jpg,121,11000000,775398007,8.203,21056


## Genres

In [38]:
g = movie_metadata[['id','genres']]
g.replace([None],[np.nan],inplace=True)
g

,id,genres
1,2,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
2,3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
3,5,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam..."
4,6,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam..."
5,11,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '..."
...,...,...
49780,1144338,[]
50074,1145358,"[{'id': 37, 'name': 'Western'}]"
50369,1146246,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam..."
50371,1146249,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam..."


In [67]:
g[g.id==83972]['genres'].values[0]

[{'id': 18, 'name': 'Drama', 'movieid': 83972},
 {'id': 53, 'name': 'Thriller', 'movieid': 83972},
 {'id': 80, 'name': 'Crime', 'movieid': 83972},
 {'id': 18, 'name': 'Drama', 'movieid': 83972}]

In [57]:
lisstrong = []

for i in set(g['id']):
    if len(g[g['id']==i]['genres'].values[0]) == 0:
        lisstrong.append(i)

len(lisstrong)

1004

In [71]:
g_list = []

listrun = set(g['id'])

for i in tqdm(listrun, total= len(listrun)):
    dataf = g[g['id']==i]['genres'].values[0]
    for y in range(len(dataf)):
        dataf[y]['movieid'] = i
        g_list.append(dataf[y])   
        
genres = pd.DataFrame(g_list)
genres.replace([None],[np.nan],inplace=True)
genres.drop_duplicates(inplace=True)
genres.sort_values(by=['movieid','id'],inplace=True)
genres['keyid'] = genres['movieid'].astype('str') + genres['id'].astype('str')
genres = genres[['keyid','movieid','name']]
genres.rename(columns={'movieid':'id','name':'genres'},inplace=True)
genres

100%|██████████| 85304/85304 [00:18<00:00, 4523.07it/s]


,keyid,id,genres
2,218,2,Drama
1,235,2,Comedy
4,280,2,Crime
3,210749,2,Romance
6,318,3,Drama
...,...,...,...
108892,114624627,1146246,Horror
108893,114624653,1146246,Thriller
108895,114624927,1146249,Horror
108896,114624953,1146249,Thriller


In [73]:
for i in ['keyid']:
    print(f'{i} duplicate = {genres[i].duplicated().sum()}')

keyid duplicate = 0


In [65]:
genres[genres['keyid'].isin(list(genres[genres['keyid'].duplicated()==True]['keyid']))]

,id,name,movieid,keyid
98792,18,Drama,83972,8397218
98795,18,Drama,83972,8397218
170791,99,Documentary,251281,25128199
170792,99,Documentary,251281,25128199
90714,18,Drama,336735,33673518
90717,18,Drama,336735,33673518
90713,28,Action,336735,33673528
90716,28,Action,336735,33673528
90715,10749,Romance,336735,33673510749
90718,10749,Romance,336735,33673510749


In [64]:
list(genres[genres['keyid'].duplicated()==True]['keyid'])

['8397218', '25128199', '33673518', '33673528', '33673510749']

## Collection

In [10]:
co = movie_metadata[['id','belongs_to_collection']]
co.replace([None],[np.nan],inplace=True)
co

,id,belongs_to_collection
1,2,"{'id': 1382526, 'name': 'Aki Kaurismäki's Prol..."
2,3,"{'id': 1382526, 'name': 'Aki Kaurismäki's Prol..."
3,5,NaN
4,6,NaN
5,11,"{'id': 10, 'name': 'Star Wars Collection', 'po..."
...,...,...
49780,1144338,NaN
50074,1145358,NaN
50369,1146246,NaN
50371,1146249,NaN


In [ ]:
for i in tqdm(set(co['id'])):
    try:
        datastr = co[co['id']==i]['belongs_to_collection'].values[0]
        datastr['movieid'] = i
        # print(datastr)
    except:
        print(f'film lỗi {i}')
# data = json.loads(f'''{datastr}''')

In [58]:
co_list = []
error_list = []

image_https = 'https://image.tmdb.org/t/p/w500'

for i in tqdm(set(co['id'])):
    try:
        datastr = co[co['id']==i]['belongs_to_collection'].values[0]
        datastr['movieid'] = i
        co_list.append(datastr)
    except:
        error_list.append(i)
        
codf = pd.DataFrame(co_list)
codf.replace([None],[np.nan],inplace=True)
codf['keyid'] = codf['movieid'].astype('str') + codf['id'].astype('str')
codf['poster_path'] = codf['poster_path'].apply(lambda x: image_https + str(x) if pd.isna(x) == False else x)
codf['backdrop_path'] = codf['backdrop_path'].apply(lambda x: image_https + str(x) if pd.isna(x) == False else x)
codf.rename(columns={'id':'collection_id','movieid':'id'},inplace=True)
codf = codf[['keyid','collection_id','id','name','poster_path','backdrop_path']]
codf.sort_values(by='collection_id',inplace=True)
codf

100%|██████████| 85304/85304 [00:19<00:00, 4293.54it/s]


,keyid,collection_id,id,name,poster_path,backdrop_path
2,1110,10,11,Star Wars Collection,https://image.tmdb.org/t/p/w500/aSrMJYmQX8kpF2...,https://image.tmdb.org/t/p/w500/zZDkgOmFMVYpGA...
8019,18180810,10,181808,Star Wars Collection,https://image.tmdb.org/t/p/w500/aSrMJYmQX8kpF2...,https://image.tmdb.org/t/p/w500/zZDkgOmFMVYpGA...
8020,18181210,10,181812,Star Wars Collection,https://image.tmdb.org/t/p/w500/aSrMJYmQX8kpF2...,https://image.tmdb.org/t/p/w500/zZDkgOmFMVYpGA...
365,189110,10,1891,Star Wars Collection,https://image.tmdb.org/t/p/w500/aSrMJYmQX8kpF2...,https://image.tmdb.org/t/p/w500/zZDkgOmFMVYpGA...
369,189510,10,1895,Star Wars Collection,https://image.tmdb.org/t/p/w500/aSrMJYmQX8kpF2...,https://image.tmdb.org/t/p/w500/zZDkgOmFMVYpGA...
...,...,...,...,...,...,...
4703,557121457328,1457328,55712,Demolition High / Uni Collection,NaN,NaN
6453,1036021457328,1457328,103602,Demolition High / Uni Collection,NaN,NaN
8349,4662721458846,1458846,466272,Once Upon a Time... in Hollywood Collection,NaN,NaN
6466,8907711459230,1459230,890771,The Black Demon Collection,NaN,NaN


In [32]:
for i in ['keyid']:
    print(f'{i} duplicate = {codf[i].duplicated().sum()}')

keyid duplicate = 0


# Creadit

In [25]:
q = '''
    with 
        metadata as
            (
                select distinct *
                from staging.stg_tmdb_json_movie_metadata stjmm 
                where request_json = '{"success": false, "status_code": 34, "status_message": "The resource you requested could not be found."}'
                order by id		
            )
    select distinct *
    from staging.stg_tmdb_json_movie_credits stjmc 
    where id not in (select distinct id from metadata)
        and request_json <> '{"success": false, "status_code": 34, "status_message": "The resource you requested could not be found."}'
    order by id 
    '''
creadit = selectdf(q,connUAT)
creadit

,id,request_json
0,2,"{""id"": 2, ""cast"": [{""adult"": false, ""gender"": ..."
1,3,"{""id"": 3, ""cast"": [{""adult"": false, ""gender"": ..."
2,5,"{""id"": 5, ""cast"": [{""adult"": false, ""gender"": ..."
3,6,"{""id"": 6, ""cast"": [{""adult"": false, ""gender"": ..."
4,11,"{""id"": 11, ""cast"": [{""adult"": false, ""gender"":..."
...,...,...
37048,100313,"{""id"": 100313, ""cast"": [{""adult"": false, ""gend..."
37049,100318,"{""id"": 100318, ""cast"": [{""adult"": false, ""gend..."
37050,100323,"{""id"": 100323, ""cast"": [{""adult"": false, ""gend..."
37051,100327,"{""id"": 100327, ""cast"": [{""adult"": false, ""gend..."


In [26]:
cast_crew_list = []

for i in tqdm((set(creadit['id']))):
    datastr = creadit[creadit['id']==i]['request_json'].values[0]
    data = json.loads(f'''{datastr}''')
    cast_crew_list.append(data)
   
creaditdf = pd.DataFrame(cast_crew_list)
creaditdf.sort_values(by='id',inplace=True)
creaditdf

100%|██████████| 37053/37053 [00:15<00:00, 2406.65it/s]


,id,cast,crew
0,2,"[{'adult': False, 'gender': 2, 'id': 54768, 'k...","[{'adult': False, 'gender': 2, 'id': 16767, 'k..."
1,3,"[{'adult': False, 'gender': 2, 'id': 4826, 'kn...","[{'adult': False, 'gender': 1, 'id': 54766, 'k..."
2,5,"[{'adult': False, 'gender': 2, 'id': 3129, 'kn...","[{'adult': False, 'gender': 0, 'id': 3116, 'kn..."
3,6,"[{'adult': False, 'gender': 2, 'id': 2880, 'kn...","[{'adult': False, 'gender': 2, 'id': 2042, 'kn..."
4,11,"[{'adult': False, 'gender': 2, 'id': 2, 'known...","[{'adult': False, 'gender': 2, 'id': 1, 'known..."
...,...,...,...
37048,100313,"[{'adult': False, 'gender': 2, 'id': 31771, 'k...","[{'adult': False, 'gender': 2, 'id': 930633, '..."
37049,100318,"[{'adult': False, 'gender': 2, 'id': 33923, 'k...","[{'adult': False, 'gender': 2, 'id': 551674, '..."
37050,100323,"[{'adult': False, 'gender': 1, 'id': 121038, '...","[{'adult': False, 'gender': 2, 'id': 138172, '..."
37051,100327,"[{'adult': False, 'gender': 2, 'id': 31771, 'k...","[{'adult': False, 'gender': 2, 'id': 551674, '..."


In [27]:
# id = tmdbid movie
cast_raw = creaditdf[['id','cast']]

castdf_list = []

listrun = creaditdf['id']

# listrun = [2]

for i in tqdm(set(listrun)):
    dataf = cast_raw[cast_raw['id']==i]['cast'].values[0]
    numb_cast = len(dataf)
    for y in range(len(dataf)):
        cast_id = dataf[y]['id']
        dataf[y]['movieid'] = i
        if dataf[y]['profile_path'] is not None:
            dataf[y]['profile_path'] = 'https://image.tmdb.org/t/p/w500' + dataf[y]['profile_path']
        castdf_list.append(dataf[y])

stg_cast = pd.DataFrame(castdf_list)
stg_cast = stg_cast[stg_cast.known_for_department == 'Acting']
print(f'''credit_id' duplicate = {stg_cast['credit_id'].duplicated().sum()}''')
stg_cast = stg_cast[['credit_id', 'movieid','id', 'cast_id', 'character']]
stg_cast.rename(columns={'movieid':'id','id':'actor_id'},inplace=True)
stg_cast

100%|██████████| 37053/37053 [00:09<00:00, 3806.42it/s]


credit_id' duplicate = 0


,credit_id,id,actor_id,cast_id,character
0,52fe420dc3a36847f8000029,2,54768,3,Taisto Kasurinen
1,52fe420dc3a36847f800002d,2,54769,4,Irmeli Pihlaja
2,52fe420dc3a36847f8000031,2,4826,5,Mikkonen
3,52fe420dc3a36847f8000035,2,54770,6,Riku
4,5e1cda97d5dbc20017f439bb,2,1177496,12,Miner
...,...,...,...,...,...
870264,52fe49d4c3a36847f81a88ad,100329,1169622,1007,Senén
870265,52fe49d4c3a36847f81a88b1,100329,967362,1008,Felisa
870267,52fe49d4c3a36847f81a88b9,100329,1023096,1010,Doctor
870268,52fe49d4c3a36847f81a88bd,100329,1169623,1011,Cura


In [30]:
ctest = stg_cast[stg_cast.index==538344]
ctest

,credit_id,id,actor_id,cast_id,character
538344,59b5a875c3a3686890001111,43211,1885525,23,"Paolo Fabbris, detto ""Paolone"""


In [31]:
stg_cast[stg_cast.actor_id == 1885525]

,credit_id,id,actor_id,cast_id,character
538344,59b5a875c3a3686890001111,43211,1885525,23,"Paolo Fabbris, detto ""Paolone"""


In [32]:
sql_table_name = 'cast'
python_table = ctest
sql = f"INSERT INTO {sql_table_name} VALUES ({','.join(['%s'] * len(python_table.columns))})"
sql

'INSERT INTO cast VALUES (%s,%s,%s,%s,%s)'

In [33]:
data = [tuple(row) for _, row in tqdm(python_table.iterrows(), total=len(python_table), desc="Preparing data")]
data

Preparing data: 100%|██████████| 1/1 [00:00<?, ?it/s]


[('59b5a875c3a3686890001111',
  43211,
  1885525,
  23,
  'Paolo Fabbris, detto "Paolone"')]

In [34]:
# id = tmdbid movie
crew_raw = creaditdf[['id','crew']]

crewdf_list = []

listrun = creaditdf['id']

# listrun = [2]

for i in tqdm(set(listrun)):
    dataf = crew_raw[crew_raw['id']==i]['crew'].values[0]
    numb_crew = len(dataf)
    for y in range(len(dataf)):
        crew_id = dataf[y]['id']
        dataf[y]['movieid'] = i
        if dataf[y]['profile_path'] is not None:
            dataf[y]['profile_path'] = 'https://image.tmdb.org/t/p/w500' + dataf[y]['profile_path']
        crewdf_list.append(dataf[y])

stg_crew = pd.DataFrame(crewdf_list)
stg_crew = stg_crew[stg_crew.known_for_department != 'Acting']
print(f'''credit_id' duplicate = {stg_crew['credit_id'].duplicated().sum()}''')
stg_crew = stg_crew[['credit_id', 'movieid','id', 'department', 'job']]
stg_crew.rename(columns={'movieid':'id','id':'crew_id'},inplace=True)
stg_crew

100%|██████████| 37053/37053 [00:09<00:00, 3804.03it/s]


credit_id' duplicate = 0


,credit_id,id,crew_id,department,job
0,52fe420dc3a36847f8000025,2,16767,Directing,Director
1,52fe420dc3a36847f800003b,2,16769,Camera,Director of Photography
2,52fe420dc3a36847f8000041,2,54766,Editing,Editor
3,52fe420dc3a36847f8000047,2,53836,Art,Production Design
4,52fe420dc3a36847f800004d,2,54771,Costume & Make-Up,Costume Design
...,...,...,...,...,...
1035818,52fe49d4c3a36847f81a8895,100329,1024245,Writing,Writer
1035819,52fe49d4c3a36847f81a88d3,100329,6525,Writing,Co-Writer
1035820,52fe49d4c3a36847f81a88cd,100329,15604,Editing,Editor
1035821,52fe49d4c3a36847f81a88c7,100329,63975,Camera,Director of Photography


In [35]:
for i in ['credit_id']:
    print(f'{i} duplicate = {stg_crew[i].duplicated().sum()}')

credit_id duplicate = 0


# Person

In [95]:
q = '''
        select distinct *
        from staging.stg_tmdb_json_person stjp 
    '''
per = selectdf(q,connPROC)
per

,id,request_json
0,1,"{""adult"": false, ""also_known_as"": [""George Wal..."
1,2,"{""adult"": false, ""also_known_as"": [""Mark Hamil..."
2,3,"{""adult"": false, ""also_known_as"": [""Harrison J..."
3,4,"{""adult"": false, ""also_known_as"": [""Carrie Fra..."
4,5,"{""adult"": false, ""also_known_as"": [""Peter Wilt..."
...,...,...
28236,36010,"{""adult"": false, ""also_known_as"": [], ""biograp..."
28237,36012,"{""adult"": false, ""also_known_as"": [], ""biograp..."
28238,36013,"{""adult"": false, ""also_known_as"": [], ""biograp..."
28239,36014,"{""adult"": false, ""also_known_as"": [""\u0419\u04..."


In [ ]:
per_list = []

image_https = 'https://image.tmdb.org/t/p/w500'

for i in tqdm((set(per['id']))):
    datastr = per[per['id']==i]['request_json'].values[0]
    data = json.loads(f'''{datastr}''')
    per_list.append(data)

perdf = pd.DataFrame(per_list)
perdf.replace([None],[np.nan],inplace=True)
perdf.replace([''],[np.nan],inplace=True)
perdf['also_known_as'] = perdf['also_known_as'].apply(lambda x: np.nan if len(x)==0 else x)
perdf.sort_values(by='id',inplace=True)
perdf = perdf[['id', 'imdb_id','name','also_known_as','gender',
                'birthday', 'deathday', 'place_of_birth',
                'known_for_department','popularity','profile_path','biography']]
perdf.rename(columns={'known_for_department':'job','also_known_as':'also_name'},inplace=True)
perdf['profile_path'] = image_https + perdf['profile_path']
perdf

100%|██████████| 28241/28241 [00:07<00:00, 3862.80it/s]


,id,imdb_id,name,also_name,gender,birthday,deathday,place_of_birth,job,popularity,profile_path,biography
0,1,nm0000184,George Lucas,[George Walton Lucas Jr. ],2,1944-05-14,NaN,"Modesto, California, USA",Directing,2.0030,https://image.tmdb.org/t/p/w500/mDLDvsx8PaZoET...,"George Walton Lucas Jr. (born May 14, 1944) is..."
1,2,nm0000434,Mark Hamill,"[Mark Hamil, Mark Richard Hamill, Patrick Will...",2,1951-09-25,NaN,"Oakland, California, USA",Acting,2.4256,https://image.tmdb.org/t/p/w500/2ZulC2Ccq1yv3p...,Mark Richard Hamill (/ˈhæməl/; born September ...
2,3,nm0000148,Harrison Ford,[Harrison J. Ford],2,1942-07-13,NaN,"Chicago, Illinois, USA",Acting,4.4896,https://image.tmdb.org/t/p/w500/zVnHagUvXkR2St...,Legendary Hollywood Icon Harrison Ford was bor...
3,4,nm0000402,Carrie Fisher,"[Carrie Frances Fisher , Кэрри Фрэнсис Фишер, ...",1,1956-10-21,2016-12-27,"Beverly Hills, Los Angeles, California, USA",Acting,1.5892,https://image.tmdb.org/t/p/w500/awb4UqzT6meD3J...,"Carrie Frances Fisher (October 21, 1956 - Dece..."
4,5,nm0001088,Peter Cushing,"[Peter Wilton Cushing, 彼得·庫辛, Питер Кушинг, Пи...",2,1913-05-26,1994-08-11,"Kenley, Surrey, England, UK",Acting,0.6526,https://image.tmdb.org/t/p/w500/if5g03wn6uvHx7...,"Peter Wilton Cushing, OBE (26 May 1913 – 11 A..."
...,...,...,...,...,...,...,...,...,...,...,...,...
28236,36010,nm0586923,Axel Milberg,NaN,2,1956-08-01,NaN,"Kiel, Germany",Acting,0.2488,https://image.tmdb.org/t/p/w500/eK1Kkda8rOZMgo...,NaN
28237,36012,nm0256797,Peter Ender,NaN,0,NaN,NaN,NaN,Acting,0.1016,NaN,NaN
28238,36013,nm0772589,Thomas Schmauser,NaN,2,1972-11-30,NaN,"Burgebrach, Germany",Acting,0.0913,https://image.tmdb.org/t/p/w500/enQxLeqpx2if9H...,NaN
28239,36014,nm0902083,Johann von Bülow,[Йохан фон Бюлов],2,1972-09-26,NaN,"Munich, Bavaria, West Germany",Acting,0.3582,https://image.tmdb.org/t/p/w500/woQtH2CnnpzAwH...,NaN


In [97]:
for i in ['id']:
    print(f'{i} duplicate = {perdf[i].duplicated().sum()}')

id duplicate = 0


In [93]:
len(perdf[perdf['id']==2]['also_name'].values[0])

7

In [94]:
perdf[perdf['id']==2]['also_name'].values[0]

['Mark Hamil',
 'Mark Richard Hamill',
 'Patrick Williams',
 'William M. Patrick',
 '马克哈米尔',
 '馬克哈米爾',
 'مارک همیل']

In [91]:
list_name = []

for i in set(perdf['id']):
    try:
        if len(perdf[perdf['id']==i]['also_name'].values[0]) > 1:
            list_name.append(i)
    except:
        pass


In [92]:
list_name

[2,
 4,
 5,
 6,
 7,
 10,
 11,
 13,
 14,
 18,
 19,
 20,
 22,
 24,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 39,
 40,
 42,
 47,
 48,
 49,
 50,
 52,
 53,
 59,
 62,
 63,
 64,
 65,
 66,
 68,
 70,
 74,
 79,
 81,
 85,
 98,
 99,
 100,
 102,
 103,
 108,
 109,
 110,
 112,
 113,
 114,
 116,
 118,
 126,
 128,
 129,
 130,
 131,
 133,
 134,
 138,
 139,
 140,
 141,
 147,
 150,
 151,
 152,
 155,
 164,
 183,
 185,
 186,
 187,
 190,
 192,
 193,
 194,
 198,
 200,
 202,
 204,
 205,
 206,
 207,
 212,
 223,
 224,
 227,
 228,
 230,
 236,
 240,
 243,
 246,
 251,
 255,
 258,
 280,
 282,
 287,
 288,
 289,
 290,
 291,
 309,
 322,
 325,
 326,
 327,
 328,
 333,
 334,
 335,
 336,
 346,
 349,
 350,
 352,
 356,
 366,
 368,
 369,
 378,
 380,
 381,
 382,
 397,
 399,
 400,
 404,
 414,
 417,
 418,
 421,
 423,
 426,
 427,
 429,
 448,
 450,
 451,
 452,
 453,
 460,
 463,
 475,
 478,
 480,
 486,
 488,
 491,
 492,
 494,
 496,
 497,
 500,
 501,
 502,
 503,
 504,
 510,
 514,
 515,
 516,
 517,
 518,
 519,
 520,
 521,
 522,
 524,
 525,

In [ ]:
'id', 'imdb_id','name','also_known_as','gender',
'birthday', 'deathday', 'place_of_birth',
'known_for_department','popularity','profile_path','biography'

+-----------------+                          +-----------------+
| Mô hình đầy đủ  |                          | Mô hình rút gọn  |
| (có tất cả biến)|                          | (bỏ 1 biến ra)   |
+-----------------+                          +-----------------+
         |                                          |
         |                                          |
         v                                          v
    Log Likelihood                             Log Likelihood
      L_full                                     L_reduced
         |                                          |
         |                                          |
         +-------------- so sánh ------------------+
                            |
                            v
        Likelihood Ratio Statistic:
            D = -2 (log(L_reduced) - log(L_full))
                            |
                            v
                D ~ Chi-square(df = 1)
                            |
                            v
       So sánh D với ngưỡng chi-square (ví dụ 3.84 với mức ý nghĩa 5%)
                            |
          +-----------------+----------------+
          |                                 |
  Nếu D > 3.84                     Nếu D < 3.84
  (biến quan trọng)                (biến không quan trọng)
          |                                 |
Giữ biến trong mô hình          Loại biến khỏi mô hình
